
CSYE7105 High Performance Parallel Machine Learning and AI

Instructor: Dr. Handan Liu

Example: Dask-ML for parallel Machine Learning


In [1]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=8,
                n_workers=1, memory_limit='10GB')
client

/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45785 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.99.249.84:45785/status,
Dashboard: http://10.99.249.84:45785/status,Workers: 1
Total threads: 8,Total memory: 9.31 GiB
Status: running,Using processes: False
Comm: inproc://10.99.249.84/64365/1,Workers: 1
Dashboard: http://10.99.249.84:45785/status,Total threads: 8
Started: Just now,Total memory: 9.31 GiB
Comm: inproc://10.99.249.84/64365/4,Total threads: 8
Dashboard: http://10.99.249.84:41799/status,Memory: 9.31 GiB
Nanny: None,


2025-10-28 15:00:12,008 - distributed.worker - ERROR - Compute Failed
Key:       ('concatenate-from-value-getitem-pairwise_distances_argmin_min-sum-3c80c256338c2c4efeaeef29972d372e', 6)
State:     executing
Task:  <Task ('concatenate-from-value-getitem-pairwise_distances_argmin_min-sum-3c80c256338c2c4efeaeef29972d372e', 6) _execute_subgraph(...)>
Exception: 'ValueError("could not convert string to float: np.str_(\'concatenate-b101f9af628e4476c48f1e57272db240\')")'
Traceback: '  File "/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/dask/utils.py", line 77, in apply\n    return func(*args, **kwargs)\n           ^^^^^^^^^^^^^^^^^^^^^\n  File "/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/sklearn/utils/_param_validation.py", line 216, in wrapper\n    return func(*args, **kwargs)\n           ^^^^^^^^^^^^^^^^^^^^^\n  File "/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/sklearn/metrics/pairwise.py", line 818, in pairwise_distances_argmin_min\n    X, Y = 

In [2]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [3]:
X, y = make_classification(n_samples=1000, random_state=0)
X[:5]

array([[-1.06377997,  0.67640868,  1.06935647, -0.21758002,  0.46021477,
        -0.39916689, -0.07918751,  1.20938491, -0.78531472, -0.17218611,
        -1.08535744, -0.99311895,  0.30693511,  0.06405769, -1.0542328 ,
        -0.52749607, -0.0741832 , -0.35562842,  1.05721416, -0.90259159],
       [ 0.0708476 , -1.69528125,  2.44944917, -0.5304942 , -0.93296221,
         2.86520354,  2.43572851, -1.61850016,  1.30071691,  0.34840246,
         0.54493439,  0.22532411,  0.60556322, -0.19210097, -0.06802699,
         0.9716812 , -1.79204799,  0.01708348, -0.37566904, -0.62323644],
       [ 0.94028404, -0.49214582,  0.67795602, -0.22775445,  1.40175261,
         1.23165333, -0.77746425,  0.01561602,  1.33171299,  1.08477266,
        -0.97805157, -0.05012039,  0.94838552, -0.17342825, -0.47767184,
         0.76089649,  1.00115812, -0.06946407,  1.35904607, -1.18958963],
       [-0.29951677,  0.75988955,  0.18280267, -1.55023271,  0.33821802,
         0.36324148, -2.10052547, -0.4380675 , -

In [4]:
param_grid = {"C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           cv=3,
                           n_jobs=8)

In [7]:
# To fit that normally, we would call
grid_search.fit(X, y)

GridSearchCV(cv=3,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             n_jobs=8,
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
                         'kernel': ['rbf', 'poly', 'sigmoid'],
                         'shrinking': [True, False]})

In [8]:
import joblib

with joblib.parallel_backend('dask'):
    grid_search.fit(X, y)

In [9]:
pd.DataFrame(grid_search.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_shrinking,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.121470,0.004329,0.014961,0.001550,0.001,rbf,True,"{'C': 0.001, 'kernel': 'rbf', 'shrinking': True}",0.502994,0.501502,0.501502,0.501999,0.000704,41
1,0.120248,0.011057,0.015578,0.001778,0.001,rbf,False,"{'C': 0.001, 'kernel': 'rbf', 'shrinking': False}",0.502994,0.501502,0.501502,0.501999,0.000704,41
2,0.078228,0.002707,0.009910,0.001658,0.001,poly,True,"{'C': 0.001, 'kernel': 'poly', 'shrinking': True}",0.502994,0.501502,0.501502,0.501999,0.000704,41
3,0.083108,0.002115,0.009078,0.001194,0.001,poly,False,"{'C': 0.001, 'kernel': 'poly', 'shrinking': Fa...",0.502994,0.501502,0.501502,0.501999,0.000704,41
4,0.180100,0.005584,0.016980,0.000534,0.001,sigmoid,True,"{'C': 0.001, 'kernel': 'sigmoid', 'shrinking':...",0.502994,0.501502,0.501502,0.501999,0.000704,41


In [10]:
grid_search.predict(X)[:5]

array([0, 1, 1, 1, 0])

In [11]:
grid_search.score(X, y)

0.983

In [12]:
import dask_ml.datasets
import dask_ml.cluster
import matplotlib.pyplot as plt

In [13]:
X, y = dask_ml.datasets.make_blobs(n_samples=10000000,
                                   chunks=1000000,
                                   random_state=0,
                                   centers=3)
X = X.persist()
X

dask.array<concatenate, shape=(10000000, 2), dtype=float64, chunksize=(1000000, 2), chunktype=numpy.ndarray>

In [14]:
km = dask_ml.cluster.KMeans(n_clusters=3, init_max_iter=2, oversampling_factor=10)
km.fit(X)

/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/dask/base.py:1105: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


ValueError: could not convert string to float: np.str_('concatenate-b101f9af628e4476c48f1e57272db240')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[::10000, 0], X[::10000, 1], marker='.', c=km.labels_[::10000],
           cmap='viridis', alpha=0.25);